In [4]:

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
import os
def convert(path):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    model_id = "openai/whisper-large-v3"

    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True, cache_dir="D:\hugging-models"
    )
    model.to(device)

    processor = AutoProcessor.from_pretrained(model_id)

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        max_new_tokens=128,
        chunk_length_s=30,
        batch_size=16,
        return_timestamps=True,
        torch_dtype=torch_dtype,
        device=device,
    )


    result = pipe(path)
    print(result)
    # os.remove(path)
    return result 


In [5]:
res = convert(r'F:\works\A-important\A-neurals\IEEE-PaperGen\I built a GPT Investment Banker using this 312 PAGE document.mp4')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\whisper\modeling_whisper.py:697: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


{'text': " This is an AI agent doing financial analysis. I used it to build an LLM-Langchain app that could analyze financial statements. In fact, it's doing it without fine-tuning on this 300-page banking annual report right now. I'm going to teach you how to build it in 5 minutes. I went down the rabbit hole to discover how to use your own documents to build AI-powered large language model apps with Langchain. And the results speak for themselves. But first of all, why should we even bother trying to use our own documents? We've already built our own large language model app with Streamlit. Why the need for docos? Well, it gives you the ultimate flexibility to leverage large language models for tasks that are specific to you. Provide meeting minutes and create a summary. You could upload an assignment outline and have a response generated. Upload an annual report and have a large language model do the financial analysis for you. In fact, that's what we're going to be doing we're goin

In [6]:
res["text"]

" This is an AI agent doing financial analysis. I used it to build an LLM-Langchain app that could analyze financial statements. In fact, it's doing it without fine-tuning on this 300-page banking annual report right now. I'm going to teach you how to build it in 5 minutes. I went down the rabbit hole to discover how to use your own documents to build AI-powered large language model apps with Langchain. And the results speak for themselves. But first of all, why should we even bother trying to use our own documents? We've already built our own large language model app with Streamlit. Why the need for docos? Well, it gives you the ultimate flexibility to leverage large language models for tasks that are specific to you. Provide meeting minutes and create a summary. You could upload an assignment outline and have a response generated. Upload an annual report and have a large language model do the financial analysis for you. In fact, that's what we're going to be doing we're going to be b

In [7]:
import summarization
content = summarization.content_generator(logit=res['text'][:1000])


ChatCompletion(id='chatcmpl-9Jz9SOf2pEs5o09DeVEKoow3DYHsi', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Title: Utilizing AI-Powered Large Language Models for Financial Analysis\n\nAbstract: \nIn this paper, we present a novel application of AI-powered large language models for financial analysis. We describe the development and implementation of an application that leverages Langchain and Streamlit to analyze financial statements, with a specific focus on banking annual reports. This innovative approach provides a unique and efficient method for financial analysis, offering significant benefits over traditional methods.\n\nI. INTRODUCTION\n\nArtificial Intelligence (AI) has revolutionized numerous sectors, with finance being no exception. The use of AI-powered large language models, such as Langchain, for financial analysis offers a new perspective on how financial data can be interpreted and utilized. This paper provides an over

In [8]:
content

ChatCompletion(id='chatcmpl-9Jz9SOf2pEs5o09DeVEKoow3DYHsi', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Title: Utilizing AI-Powered Large Language Models for Financial Analysis\n\nAbstract: \nIn this paper, we present a novel application of AI-powered large language models for financial analysis. We describe the development and implementation of an application that leverages Langchain and Streamlit to analyze financial statements, with a specific focus on banking annual reports. This innovative approach provides a unique and efficient method for financial analysis, offering significant benefits over traditional methods.\n\nI. INTRODUCTION\n\nArtificial Intelligence (AI) has revolutionized numerous sectors, with finance being no exception. The use of AI-powered large language models, such as Langchain, for financial analysis offers a new perspective on how financial data can be interpreted and utilized. This paper provides an over

In [9]:
len(res['text'] )

7011

In [10]:
def create_chunks(text, timestamps):
    chunks = []
    for i in range(len(timestamps)):
        start_time, end_time = timestamps[i]['timestamp']
        chunk_text = text[start_time:end_time]
        chunks.append({'timestamp': (start_time, end_time), 'text': chunk_text})
    return chunks

# Example usage
text = "This is an AI agent doing financial analysis. I used it to build an LLM-Langchain app that could analyze financial statements. In fact, it's doing it without fine-tuning on this 300-page banking annual report right now. I'm going to teach you how to build it in 5 minutes. I went down the rabbit hole to discover how to use your own documents to build AI-powered large language model apps with Langchain. And the results speak for themselves. But first of all, why should we even bother trying to use our own documents? We've already built our own large language model app with Streamlit. Why the need for docos? Well, it gives you the ultimate flexibility to leverage large language models for tasks that are specific to you. Provide meeting minutes and create a summary. You could upload an assignment outline and have a response generated. Upload an annual report and have a large language model do the financial analysis for you. In fact, that's what we're going to be doing we're going to be building our very own personal ai powered investment banker using streamlit and our own annual report to get started we first need to install some dependencies namely langchain openai streamlit tick token chroma db pi pdf and crypto dome while we're at it we're going to need to get an openai api key it's optional to use the openai service you could substitute substitute it out for a Lama CPP or one of the Hugging Face hosted models. If you want a vid on that, let me know. Time to create our app. First up, in a new Python file called app.py, we're going to import some initial dependencies. OS is going to be used to set the API key. From langchain.llms, we'll import the OpenAI service. This will be the core LLM we use. We'll also bring in Streamlit while we're at it. Once that's done, we can set the API key and create the OpenAI service as an LLM. If we wanted to, we could tune the temperature depending on how creative or objective we wanted our responses. We also need somewhere to pass the prompt. We can use the Streamlit.TextInput element to do this. If our user hits enter, we need a way to send the prompt to the LLM. Let's throw in an if statement here and write out the text to the screen using streamlit.write. To start the app it's relatively straightforward, just write streamlit run app.py or whatever your python file is named. Side note, all this code is going to be available in the description below so you can give it a crack yourself. This gives us a baseline app and uses open AI to generate responses. But there's one key issue here, we haven't gone and used our own documents yet. Let's fix that but before we do, a word from our sponsors. Me. If you'd like to get up and running with Python for machine learning, head over to go.coursesfromnick.com forward slash Python, where you can take my end-to-end tech fundamentals course. Or if you want to dive straight into the deep end, you can check out my full stack machine learning course at this link here. And use YouTube 50 to get 50% off right now. Back to the video. We need At the top of our script, we're going to bring in PyPDFLoader from langchain.documentloaders and Chroma from langchain.vectorstores. PyPDFLoader is used to load and pass a PDF into memory. Chroma is a vector store that is critical to using your own documents. What actually happens, as far as I know, is your document is tokenized and loaded into Chroma for querying later on. This allows you to perform similarity search using similarity metrics like Euclidean distance, similarity metrics like Euclidean distance. Also similar to how Pinecone works. We'll do this in a sec. All right, time to load up our document. Using the PyPDF loader class, we're going to upload a banking annual report. I've got the files stored in the same place as my Python script, so I can just pass the name of the document direct to the class. If you wanted to use your own document, you could sub in the name of the file here. In this case, it's got to be a PDF, be a PDF but there are other loaders available inside of Langchain. We can then load the document"
create_chunks(text)

TypeError: create_chunks() missing 1 required positional argument: 'timestamps'

In [ ]:
def Great(input):
    print(input)
holder = Great()
holder("ki")

TypeError: Great() missing 1 required positional argument: 'input'

In [ ]:
import os
from openai import AzureOpenAI

load_dotenv()

client = AzureOpenAI(
  azure_endpoint = "https://openai-demetrius.openai.azure.com/", 
  api_key= '99e288a8e7eb48fc91de6cc1f8e991d2',  
  api_version="2024-02-15-preview"
)

logits = "tell me joke on college"
message_text = [{"role": "user", "content": logits}, {"role":"system","content":"You are an AI assistant that makes the Funny jokes based on content"}]

completion = client.chat.completions.create(
  model="gpt4-demetrius", # model = "deployment_name"
  messages = message_text,
  temperature=0,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)
print(completion.choices[0].message.content)



Why don't college students throw parties?

Because they can't even clean up their own syllabus!


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader(res["chunks"])

# Split the text into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(loader.load())

RuntimeError: Error loading [{'timestamp': (0.08, 2.36), 'text': ' This is an AI agent doing financial analysis.'}, {'timestamp': (2.36, 5.84), 'text': ' I used it to build an LLM-Langchain app that could analyze financial statements.'}, {'timestamp': (5.84, 9.92), 'text': " In fact, it's doing it without fine-tuning on this 300-page banking annual report right"}, {'timestamp': (9.92, 10.92), 'text': ' now.'}, {'timestamp': (10.92, 12.04), 'text': " I'm going to teach you how to build it in 5 minutes."}, {'timestamp': (12.04, 15.84), 'text': ' I went down the rabbit hole to discover how to use your own documents to build AI-powered'}, {'timestamp': (15.84, 18.4), 'text': ' large language model apps with Langchain.'}, {'timestamp': (18.4, 25.8), 'text': " And the results speak for themselves. But first of all, why should we even bother trying to use our own documents? We've already built our own large language model app with Streamlit."}, {'timestamp': (25.8, 26.8), 'text': ' Why the ne